In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import datetime as dt

import sqlalchemy as db
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify, render_template

import pandas as pd

engine = create_engine("sqlite:///movie_ratings_db.sqlite")

Base = automap_base()

Base.prepare(autoload_with=engine)

Links = Base.classes.links
Movies = Base.classes.movies
Ratings = Base.classes.ratings
Tags = Base.classes.tags

print(Base.classes.keys())


['links', 'movies', 'ratings', 'tags']


In [2]:
# https://www.geeksforgeeks.org/sqlalchemy-orm-conversion-to-pandas-dataframe/#

links_df = pd.read_sql_query(
    sql = db.select([Links.movieId,
                     Links.imdbId,
                     Links.tmdbId]),
    con = engine
)

print(len(links_df)) # Length should be 9742
links_df.head()

# We can probably ignore this table

9742


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [3]:
movies_df = pd.read_sql_query(
    sql = db.select([Movies.movieId,
                     Movies.title,
                     Movies.genres]),
    con = engine
)

print(len(movies_df)) # Length should be 9742
movies_df.head()

9742


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_df = pd.read_sql_query(
    sql = db.select([Ratings.userId,
                     Ratings.movieId,
                     Ratings.rating,
                     Ratings.timestamp]),
    con = engine
)

print(len(ratings_df)) # Length should be 100836
ratings_df.head()

100836


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [33]:
tags_df = pd.read_sql_query(
    sql = db.select([Tags.userId,
                     Tags.movieId,
                     Tags.tag,
                     Tags.timestamp]),
    con = engine
)

print(len(tags_df)) # Length should be 3683
tags_df.head()

3683


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
# Data preprocessing

In [30]:
movies_df_copy = movies_df.copy()
new = movies_df_copy["genres"].str.split("|",expand=True)
for i in new:
    movies_df_copy[f"genre{i + 1}"] = new[i]
movies_df_copy.drop(columns=["genres"], inplace=True)
movies_df_copy.head()

,movieId,title,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10
0,1,Toy Story (1995),Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
1,2,Jumanji (1995),Adventure,Children,Fantasy,None,None,None,None,None,None,None
2,3,Grumpier Old Men (1995),Comedy,Romance,None,None,None,None,None,None,None,None
3,4,Waiting to Exhale (1995),Comedy,Drama,Romance,None,None,None,None,None,None,None
4,5,Father of the Bride Part II (1995),Comedy,None,None,None,None,None,None,None,None,None


In [31]:
movies_df_copy_2 = movies_df.copy()
new2 = movies_df_copy_2["genres"].str.split("|", expand=True)
movies_df_copy_2["genre"] = new2[0]
movies_df_copy_2.drop(columns=["genres"], inplace=True)
movies_df_copy_2.head()

,movieId,title,genre
0,1,Toy Story (1995),Adventure
1,2,Jumanji (1995),Adventure
2,3,Grumpier Old Men (1995),Comedy
3,4,Waiting to Exhale (1995),Comedy
4,5,Father of the Bride Part II (1995),Comedy


In [41]:
toy_story = tags_df.loc[tags_df["movieId"] == 1]
len(toy_story)

3

In [43]:
toy_story_2 = ratings_df.loc[ratings_df["movieId"] == 1]
toy_story_2.head(25)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
516,5,1,4.0,847434962
874,7,1,4.5,1106635946
1434,15,1,2.5,1510577970
1667,17,1,4.5,1305696483
1772,18,1,3.5,1455209816
2274,19,1,4.0,965705637
3219,21,1,3.5,1407618878
4059,27,1,3.0,962685262
4879,31,1,5.0,850466616


In [58]:
movies_to_merge = movies_df_copy_2.copy()
tags_to_merge = tags_df.copy()

movies_tags = pd.merge(tags_to_merge, movies_to_merge, on="movieId", how="right")
movies_tags.head()

,userId,movieId,tag,timestamp,title,genre
0,336.0,1,pixar,1.139046e+09,Toy Story (1995),Adventure
1,474.0,1,pixar,1.137207e+09,Toy Story (1995),Adventure
2,567.0,1,fun,1.525286e+09,Toy Story (1995),Adventure
3,62.0,2,fantasy,1.528844e+09,Jumanji (1995),Adventure
4,62.0,2,magic board game,1.528844e+09,Jumanji (1995),Adventure


In [59]:
len(movies_tags)

11853

In [64]:
movies_tags_modified = movies_tags.dropna()
len(movies_tags_modified)

3683

In [66]:
ratings_to_merge = ratings_df.copy()
ratings_to_merge = ratings_to_merge.drop(columns=["timestamp"])
movies_tags_to_merge = movies_tags_modified.copy()
movies_tags_to_merge = movies_tags_to_merge.drop(columns=["timestamp"])

movies_tags_ratings = pd.merge(ratings_to_merge, movies_tags_to_merge, on=["userId","movieId"], how="left")
print(len(movies_tags_ratings))
mtr_modified = movies_tags_ratings.dropna()
print(len(mtr_modified))
mtr_modified.head(25)

102677
3476


,userId,movieId,rating,tag,title,genre
241,2,60756,5.0,funny,Step Brothers (2008),Comedy
242,2,60756,5.0,Highly quotable,Step Brothers (2008),Comedy
243,2,60756,5.0,will ferrell,Step Brothers (2008),Comedy
252,2,89774,5.0,Boxing story,Warrior (2011),Drama
253,2,89774,5.0,MMA,Warrior (2011),Drama
254,2,89774,5.0,Tom Hardy,Warrior (2011),Drama
258,2,106782,5.0,drugs,"Wolf of Wall Street, The (2013)",Comedy
259,2,106782,5.0,Leonardo DiCaprio,"Wolf of Wall Street, The (2013)",Comedy
260,2,106782,5.0,Martin Scorsese,"Wolf of Wall Street, The (2013)",Comedy
1025,7,48516,1.0,way too long,"Departed, The (2006)",Crime


In [68]:
print(len(mtr_modified["title"].unique()))

1464


In [57]:
movies_tags_to_merge.head()

,userId,movieId,tag,title,genre
0,2,60756,funny,Step Brothers (2008),Comedy
1,2,60756,Highly quotable,Step Brothers (2008),Comedy
2,2,60756,will ferrell,Step Brothers (2008),Comedy
3,2,89774,Boxing story,Warrior (2011),Drama
4,2,89774,MMA,Warrior (2011),Drama


In [76]:
tags_grouped = tags_df.groupby(["movieId"]).agg({"tag": ",".join})
print(len(tags_grouped))
tags_grouped.head()

1572


,tag
movieId,
1,"pixar,pixar,fun"
2,"fantasy,magic board game,Robin Williams,game"
3,"moldy,old"
5,"pregnancy,remake"
7,remake


In [74]:
tags_df.nunique()

userId         58
movieId      1572
tag          1589
timestamp    3411
dtype: int64

In [ ]:
# ML model (supervised learning - logistic regression?)